In [1]:
import pandas as pd
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [2]:
mapbox_access_token = open('mapbox_token.txt').read()

In [3]:
collected = pd.read_csv('Collected.csv')
print(collected.head())

   Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0 Location    Latitude, Longitude  \
0             0             0           0    Kilju  40.963361, 129.320537   
1             1             1           1  Hamhung  39.991616, 127.612627   
2             2             2           2     Nara  34.696509, 135.830573   
3             3             3           3  Hamhung  39.991616, 127.612627   
4             4             4           4   Wonsan  39.153670, 127.446306   

          Date Name_en Name_ko   Latitude   Longitude Start_date End_date  \
0  6/1/1911 -     ImOI     임옥인  40.963361  129.320537   6/1/1911      NaN   
1       1931 -    ImOI     임옥인  39.991616  127.612627       1931      NaN   
2    3/1932 -     ImOI     임옥인  34.696509  135.830573     3/1932      NaN   
3        1935-    ImOI     임옥인  39.991616  127.612627       1935      NaN   
4      1937 -     ImOI     임옥인  39.153670  127.446306       1937      NaN   

  Full_Name_en          colors  
0     Im Ok-in  rgb(19,189,39)  
1     Im

In [4]:
# Create a dictionary of english name : korean name
en_name = ['Im Ok-in', 'Shim Hoon', 'Lee Kwang-soo', 'Kim Myung-soon', 'Im Soon-deuk', 'Kim Nam-cheon', 'Mo Yoon-sook', 'Ju Yo-han', 'Lee Sun-hee', 'Cha Mi-ri-sa', 'Yeom Sang-seob', 'No Chun-myeong', 'Kang Kyung-ae', 'Baek Shin-ae', 'Han Mu-sook', 'Park Hwa-seong', 'Choi Jeong-hee', 'Choi Seung-hee', 'Na Hye-seok', 'Lee Sang', 'Kim Sa-ryang', 'Ji Ha-ryeon']
ko_name = list(collected['Name_ko'].unique())
ko_en_name = {key:value for key, value in zip(ko_name, en_name)}

In [5]:
# Add a column of full english names 'Full_Name_en'
collected['Full_Name_en'] = collected['Name_ko'].apply(lambda x: ko_en_name.get(x))
print(collected.head())

   Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0 Location    Latitude, Longitude  \
0             0             0           0    Kilju  40.963361, 129.320537   
1             1             1           1  Hamhung  39.991616, 127.612627   
2             2             2           2     Nara  34.696509, 135.830573   
3             3             3           3  Hamhung  39.991616, 127.612627   
4             4             4           4   Wonsan  39.153670, 127.446306   

          Date Name_en Name_ko   Latitude   Longitude Start_date End_date  \
0  6/1/1911 -     ImOI     임옥인  40.963361  129.320537   6/1/1911      NaN   
1       1931 -    ImOI     임옥인  39.991616  127.612627       1931      NaN   
2    3/1932 -     ImOI     임옥인  34.696509  135.830573     3/1932      NaN   
3        1935-    ImOI     임옥인  39.991616  127.612627       1935      NaN   
4      1937 -     ImOI     임옥인  39.153670  127.446306       1937      NaN   

  Full_Name_en          colors  
0     Im Ok-in  rgb(19,189,39)  
1     Im

In [6]:
#Update Collected.csv
#collected.to_csv('Collected.csv')

In [7]:
# Create dictionary of author names for dropdown
authors_collected = [{'label' : name, 'value': name} for name in collected['Full_Name_en'].unique()]

In [8]:
#for whatever authors are selected, create a list of random colors generated
#make sure that the colors don't overlap
#random color generator? 
#print(index) before the graph with the colors and 
# marker(color - list I'd have created)

In [9]:
# random color generator 
import random 

# generate random colors
def color_generator(authors):
    color_dict = {}
    for i in range(len(authors)):
        r = random.randint(0,255)
        b = random.randint(0,255)
        g = random.randint(0,255)
        color = 'rgb({r},{b},{g})'.format(r=r,b=b,g=g)
        if color not in color_dict:
            color_dict[authors[i]] = color
    return color_dict

In [10]:
# Created 'colors' column in Collected.csv
color_dict = color_generator(list(collected['Full_Name_en'].unique()))
collected['colors'] = collected['Full_Name_en'].apply(lambda x: color_dict.get(x))
#collected.to_csv('Collected.csv')

In [11]:
# Dash App
from jupyter_dash import JupyterDash
app = JupyterDash(__name__)

In [12]:
# app layout using scattermapbox 
app.layout = html.Div([
    html.Div([dcc.Dropdown(
        id = 'author_dropdown',
        placeholder = 'Select poet(s)',
        value = 'Na Hye-seok',
        multi = True,
        options = authors_collected)]),
    html.Div([
    html.Button('Next ▶', id='next_button'),
    #include index 
    dcc.Graph(id = 'map', config={'displayModeBar': False, 'scrollZoom': True})])
    ])

In [13]:
#ISSUES 
# na hye seok shows up only after the dropdown values are changed 
# need to include index of colors - authors
# the size of the world map needs to only be the world 
# the world map should have bounds/domains (no multiverse // look at dcc.Graph(dict(CONFIG))

In [14]:
# Callback: author_dropdown --> update_graph
@app.callback(
    Output(component_id = 'map', component_property = 'figure'),
    [Input(component_id = 'author_dropdown', component_property = 'value')]
)
def update_graph(author_chosen):
    df = collected[collected['Full_Name_en'].isin(list(author_chosen))].reset_index()
    fig = [go.Scattermapbox(
                lat=df['Latitude'],
                lon=df['Longitude'],
                mode='markers',
                marker={
                    'color': df['colors'],  # Assign color based on the 'Name_ko' column value
                    'size': 9
                },
                #unselected={'marker' : {'opacity':1}},
                #selected={'marker' : {'opacity':0.5, 'size':10}},
                hoverinfo='text',
                hovertext=df['Location']
        )]
    
    # Return figure
    return {
        'data' : fig,
        'layout': go.Layout(
            #clickmode = 'select',
            title = '20th Century Women Writers Itinerary',
            hovermode = 'closest',
            mapbox = dict(
                accesstoken=mapbox_access_token,
                style = 'light', 
                center = dict(lat = 0, lon = 0),
                zoom = 1
        )
            
    )
        
}

In [16]:
# specify port if error about port number being used already 
if __name__ == '__main__':
    app.run_server(port = 8056)

Dash app running on http://127.0.0.1:8056/


In [15]:
#FUTURE GOALS 
# Callback --> click the marker and display PURPOSE + location (use BOOTSTRAP) 
# Callback --> NEXT BUTTON (add trace; line?) --> should this only be for 1 author each time? 
# chained callback (gender, year, location)
# css
# display photos and videos
# Multi page
   # infos
   # georeference old korean map of certain years 
# render to publish website 
# webscraping 